In [2]:
Rb = 7.568067737283432
RbO = 8.36675464062834
RbD = 7.568067737283431
Delta_end = 28.84624072309878

from bloqade.analog import load, save, get_capabilities, piecewise_linear
from bloqade.analog.atom_arrangement import Square

import numpy as np
import os
import matplotlib.pyplot as plt

from bokeh.io import output_notebook

lattice_spacing = 5.0

# get capabilities for Aquila
aquila_capabilities = get_capabilities()

C6=float(aquila_capabilities.capabilities.rydberg.c6_coefficient)
Omega_max= float(aquila_capabilities.capabilities.rydberg.global_.rabi_frequency_max) #2*np.pi*4 #

Rba=np.sqrt(2*np.sqrt(2))
Rb=Rba*lattice_spacing

Delta_end= (C6)/Rb**6 #final detuning 2*np.pi*11

RbO=(C6/Omega_max)**(1/6)
RbD=(C6/Delta_end)**(1/6)

In [3]:
def n_probable_counts(report, n=1):
    return list(report.counts()[0].keys())[:n]

def remove_site_indices(bitstring):
    return [i for i, bit in enumerate(bitstring) if bit == '0']

def post_processing(result, lattice, lattice_spacing):
    report = result.report()
    bitstring = n_probable_counts(report)[0]
    indices = remove_site_indices(bitstring)
    
    children = lattice.children()
    locs = [children[i].position for i in range(len(children))]
    filled = [i not in indices for i in range(len(children))]
    
    new_lattice = Square(0, lattice_spacing=lattice_spacing).add_position(locs, filled).remove_vacant_sites()
    return new_lattice, indices

def mis(lattice, lattice_spacing, Omega_max, Delta_end, global_indices=None, portfolios=None):
    if portfolios is None:
        portfolios = []
    if global_indices is None:
        global_indices = list(range(len(lattice.children())))

    if len(global_indices) == 0:
        return portfolios

    lattice.show()

    # Setup annealing pulse
    T_max = 4
    sweep_time = 0.8 * T_max
    ramp_time = 0.1 * T_max
    durations = [ramp_time, sweep_time, ramp_time]

    RabiProtocol = piecewise_linear(durations, [0.0, Omega_max, Omega_max, 0.0])
    DeltaProtocol = piecewise_linear(durations, [-Delta_end, -Delta_end, Delta_end, Delta_end])

    program = (
        lattice.rydberg.rabi.amplitude.uniform.apply(RabiProtocol)
        .detuning.uniform.apply(DeltaProtocol)
    )

    result = program.bloqade.python().run(100, interaction_picture=True)
    new_lattice, removed_local_indices = post_processing(result, lattice, lattice_spacing)

    # Map local indices back to global ones
    removed_global_indices = [global_indices[i] for i in removed_local_indices]
    portfolios.append(removed_global_indices)

    # Prepare global indices for next iteration
    new_global_indices = [global_indices[i] for i in range(len(global_indices)) if i not in removed_local_indices]

    if len(new_global_indices) == 0:
        return portfolios
    else:
        return mis(new_lattice, lattice_spacing, Omega_max, Delta_end, new_global_indices, portfolios)


In [4]:
rng = np.random.default_rng(1234)
lattice_init = Square(4, lattice_spacing=lattice_spacing).apply_defect_density(0.3, rng=rng).remove_vacant_sites()
portfolios = mis(lattice_init, lattice_spacing, Omega_max, Delta_end)
print("MIS portfolios (global indices):", portfolios)

MIS portfolios (global indices): [[0, 4, 8, 11], [2, 3, 7, 9], [1, 5, 10], [6]]
